## HamLib Simulation Observable Benchmark Tests - WIP

This notebook is being used to flesh out the details of integrating the HamLib utility module, the HamLib kernel function, and the newly developed functions for generating groups of commuting terms.

From any specific Hamiltonian, we create a base evolution circuit with 0 - N Trotter steps, initialize the circuit, and append the necessary basis rotations before measuring and computing the values of energy and other related observables.

At a later point, the execution code and plotting code will also be integrated into the benchmark framework. For now, however, the focus is on developing the code necessary to evaluate the performance of observable execution using various optimization options.


### Setup Dependencies and Initialize Modules

This cell is used to pull in dependencies from standard libraries, the Hamlib helper functions, and the newly created observables module (in WIP).
Initialize the modules so a re-run from the beginning is repeatable.

In [1]:
import numpy as np
from math import sin, cos, pi
import time

# Import Qiskit and Qiskit Pauli operator classes
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit import QuantumCircuit, transpile

# Initialize simulator backend
from qiskit_aer import Aer
backend = Aer.get_backend('qasm_simulator')

# Import Hamlib helper functions
import hamlib_simulation_kernel, hamlib_utils

# Restore default parameters in HamLib kernel module
hamlib_simulation_kernel.initialize()

# Import Observable helper functions
import sys
sys.path.insert(1, "WIP_benchmarks")
import observables


### Configure Options for Execution

In [2]:
# Qubit width of the Hamiltonian
num_qubits = 6

# Parameters of Trotterized simulation
K = 1
t = 0.05

# for debugging Hamlib helper functions
hamlib_simulation_kernel.verbose = False
hamlib_utils.verbose = False


### Specify a Hamiltonian for Testing
Choose a Hamiltonian from HamLib or create a custom Hamiltonian


In [3]:
# create the HamLib Simulation kernel, and the associated Hamiltonian operator

# Specify the desired Hamiltonian from HamLib 
# Set the hamiltonian_name: 'TFIM', 'Fermi-Hubbard-1D', 'Bose-Hubbard-1D', 'Heisenberg' or 'Max3Sat'
# If no name, it means we use an explicitly specified Hamiltonian
hamiltonian_name = ''

########### HamLib Hamiltonians

if hamiltonian_name == 'TFIM':
    hamlib_simulation_kernel.global_h = 2
    hamlib_simulation_kernel.global_pbc_val = 'pbc'

if hamiltonian_name == 'Fermi-Hubbard-1D':
    hamlib_simulation_kernel.global_h = 2
    hamlib_simulation_kernel.global_pbc_val = 'pbc'

if hamiltonian_name == 'Bose-Hubbard-1D':
    hamlib_simulation_kernel.global_pbc_val = 'nonpbc'
    hamlib_simulation_kernel.global_U = 10
    hamlib_simulation_kernel.global_enc = 'gray'
    
########### Explicit Hamiltonians

# classical simple Ising is ZZ
# TFIM ZZ + X  is transverse field
# + longitudinal field -> ZZ, X, and Z
def get_ising_hamiltonian(L, J, h, alpha=0):

    # List of Hamiltonian terms as 3-tuples containing
    # (1) the Pauli string,
    # (2) the qubit indices corresponding to the Pauli string,
    # (3) the coefficient.
    ZZ_tuples = [("ZZ", [i, i + 1], -J) for i in range(0, L - 1)]
    Z_tuples = [("Z", [i], -h * sin(alpha)) for i in range(0, L)]
    X_tuples = [("X", [i], -h * cos(alpha)) for i in range(0, L)]

    # We create the Hamiltonian as a SparsePauliOp, via the method
    # `from_sparse_list`, and multiply by the interaction term.
    hamiltonian = SparsePauliOp.from_sparse_list([*ZZ_tuples, *Z_tuples, *X_tuples], num_qubits=L)
    return hamiltonian.simplify()

# create explicit Hamiltonian
if hamiltonian_name == '':
    H = get_ising_hamiltonian(L=num_qubits, J=0.2, h=1.2, alpha=pi / 8)
    H_terms = H.to_list()
    
    # hamiltonian = H_terms
    # ham_terms = hamiltonian
    ham_terms = H_terms
    qc = None


### Create a Hamiltonian Simulation Circuit
Initialize the circuit to a known initial state, append 'K' Trotter steps for total time 't', followed by basis rotation gates.

In [4]:
# create the HamLib Simulation kernel, from the current Hamiltonian.
# We use a different method for kernel creation with HamLib than with custom Hamiltonians

########### create Trotterized evolution circuit for HamLib Hamiltonians

# create Hamiltonian evolution circuit from HamLib
if hamiltonian_name != 'TFIM':
    init_state = "checkerboard"

    # this function reads the HamLib file content for the specified Hamiltonian
    hamlib_simulation_kernel.get_hamiltonian_info(hamiltonian_name=hamiltonian_name, init_state=init_state)
            
    qc, bitstring, ham_op = hamlib_simulation_kernel.HamiltonianSimulation(
        num_qubits, 
        K=K, t=t,
        hamiltonian = hamiltonian_name, 
        init_state = init_state,
        append_measurements = False,
        method = 1, 
    )

    # convert SparsePauliOp to list form
    ham_terms = ham_op.to_list()
    
########### create Trotterized evolution circuit for custom Hamiltonian
else:
    pass

print(f"... Hamiltonian name = {hamiltonian_name}")
print(ham_terms)
print("")

print(f"... Trotterized Circuit, K={K}, t={t}")
if num_qubits < 10:
    print(qc)
    print("")


... Hamiltonian name = 
[('IIIIZZ', (-0.2+0j)), ('IIIZZI', (-0.2+0j)), ('IIZZII', (-0.2+0j)), ('IZZIII', (-0.2+0j)), ('ZZIIII', (-0.2+0j)), ('IIIIIZ', (-0.4592201188381077+0j)), ('IIIIZI', (-0.4592201188381077+0j)), ('IIIZII', (-0.4592201188381077+0j)), ('IIZIII', (-0.4592201188381077+0j)), ('IZIIII', (-0.4592201188381077+0j)), ('ZIIIII', (-0.4592201188381077+0j)), ('IIIIIX', (-1.1086554390135441+0j)), ('IIIIXI', (-1.1086554390135441+0j)), ('IIIXII', (-1.1086554390135441+0j)), ('IIXIII', (-1.1086554390135441+0j)), ('IXIIII', (-1.1086554390135441+0j)), ('XIIIII', (-1.1086554390135441+0j))]

... Trotterized Circuit, K=1, t=0.05
None



### Compute Energy of the Hamiltonian - Unoptimized

Here, we convert the Hamiltonian to a set of circuits, using commuting groups if specified,
and execute the circuits in order to compute the expectation value 

In [5]:
# Flag to control optimize by use of commuting groups
use_commuting_groups = False

# Create circuits from the Hamiltonian 
circuits = observables.create_circuits_for_hamiltonian(num_qubits, qc, ham_terms, use_commuting_groups)

for circuit in circuits:
    print(circuit)
    print(circuit[0])

N = 3

print(f"\n************ Compute energy of the Hamiltonian {N} times")

# Calculate the total energy
for i in range(N):
    ts = time.time()
    
    # Compile and execute the circuits
    transpiled_circuits = transpile([circuit for circuit, group in circuits], backend)

    print("")
    print(f"... transpilation time = {round(time.time()-ts, 3)}")
    
    # Execute all of the circuits to obtain array of result objects
    results = backend.run(transpiled_circuits).result()

    # Compute the total energy for the Hamiltonian
    term_contributions = {}
    total_energy = observables.calculate_expectation(num_qubits, results, circuits,
                                    term_contributions=term_contributions)

    print(f"... total execution time = {round(time.time()-ts, 3)}")
    print(f"Total Energy: {total_energy}")
    print(f"Term Contributions: {term_contributions}")
    
print("")


******** creating circuits from Hamiltonian:
  ... IIIIZZ, (-0.2+0j)
  ... IIIZZI, (-0.2+0j)
  ... IIZZII, (-0.2+0j)
  ... IZZIII, (-0.2+0j)
  ... ZZIIII, (-0.2+0j)
  ... IIIIIZ, (-0.4592201188381077+0j)
  ... IIIIZI, (-0.4592201188381077+0j)
  ... IIIZII, (-0.4592201188381077+0j)
  ... IIZIII, (-0.4592201188381077+0j)
  ... IZIIII, (-0.4592201188381077+0j)
  ... ZIIIII, (-0.4592201188381077+0j)
  ... IIIIIX, (-1.1086554390135441+0j)
  ... IIIIXI, (-1.1086554390135441+0j)
  ... IIIXII, (-1.1086554390135441+0j)
  ... IIXIII, (-1.1086554390135441+0j)
  ... IXIIII, (-1.1086554390135441+0j)
  ... XIIIII, (-1.1086554390135441+0j)

... constructed 17 circuits for this Hamiltonian.
(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001A5121299D0>, [('IIIIZZ', (-0.2+0j))])
         ░ ┌─┐               
   q_0: ─░─┤M├───────────────
         ░ └╥┘┌─┐            
   q_1: ─░──╫─┤M├────────────
         ░  ║ └╥┘┌─┐         
   q_2: ─░──╫──╫─┤M├─────────
         ░  ║  ║ └╥┘┌─┐      
  

### Compute Energy of the Hamiltonian - Optimized

Here, we convert the Hamiltonian to a set of circuits, using commuting groups if specified,
and execute the circuits in order to compute the expectation value 

In [6]:
# Flag to control optimize by use of commuting groups
use_commuting_groups = True

# Create circuits from the Hamiltonian 
circuits = observables.create_circuits_for_hamiltonian(num_qubits, qc, ham_terms, use_commuting_groups)

for circuit in circuits:
    print(circuit)
    print(circuit[0])

N = 3

print(f"\n************ Compute energy of the Hamiltonian {N} times")

# Calculate the total energy
for i in range(N):
    ts = time.time()
    
    # Compile and execute the circuits
    transpiled_circuits = transpile([circuit for circuit, group in circuits], backend)

    print("")
    print(f"... transpilation time = {round(time.time()-ts, 3)}")
    
    # Execute all of the circuits to obtain array of result objects
    results = backend.run(transpiled_circuits).result()

    # Compute the total energy for the Hamiltonian
    term_contributions = {}
    total_energy = observables.calculate_expectation(num_qubits, results, circuits,
                                    term_contributions=term_contributions)

    print(f"... total execution time = {round(time.time()-ts, 3)}")
    print(f"Total Energy: {total_energy}")
    print(f"Term Contributions: {term_contributions}")
    
print("")


******** creating commuting groups for the Hamiltonian and circuits from the groups:
Group 1:
  IIIIZZ: (-0.2+0j)
  IIIZZI: (-0.2+0j)
  IIZZII: (-0.2+0j)
  IZZIII: (-0.2+0j)
  ZZIIII: (-0.2+0j)
  IIIIIZ: (-0.4592201188381077+0j)
  IIIIZI: (-0.4592201188381077+0j)
  IIIZII: (-0.4592201188381077+0j)
  IIZIII: (-0.4592201188381077+0j)
  IZIIII: (-0.4592201188381077+0j)
  ZIIIII: (-0.4592201188381077+0j)
Group 2:
  IIIIIX: (-1.1086554390135441+0j)
  XIIIII: (-1.1086554390135441+0j)
  IIIIXI: (-1.1086554390135441+0j)
  IIIXII: (-1.1086554390135441+0j)
  IIXIII: (-1.1086554390135441+0j)
  IXIIII: (-1.1086554390135441+0j)

... constructed 2 circuits for this Hamiltonian.
(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001A512153D90>, [('IIIIZZ', (-0.2+0j)), ('IIIZZI', (-0.2+0j)), ('IIZZII', (-0.2+0j)), ('IZZIII', (-0.2+0j)), ('ZZIIII', (-0.2+0j)), ('IIIIIZ', (-0.4592201188381077+0j)), ('IIIIZI', (-0.4592201188381077+0j)), ('IIIZII', (-0.4592201188381077+0j)), ('IIZIII', (-0.459

### Compute expectation value of other observables.
Note that this only works for the Ising model.

In [7]:
# Define additional Hamiltonian terms for other Ising observables 
H_terms_spin_correlation = observables.swap_pauli_list([(0.2,'IIIIZZ'), (0.2,'IIIZZI'), (0.2,'IIZZII'), (0.2,'IZZIII'), (0.2,'ZZIIII')])
H_terms_magnetization = observables.swap_pauli_list([(1,'IIIIIZ'), (1,'IIIIZI'), (1,'IIIZII'), (1,'IIZIII'), (1,'IZIIII'), (1, 'ZIIIII')])

spin_correlation = observables.calculate_expectation_from_contributions(H_terms_spin_correlation, term_contributions)
print(spin_correlation)

magnetization = observables.calculate_expectation_from_contributions(H_terms_magnetization, term_contributions)
print(magnetization)


1.0
6.0
